In [1]:
"""
Read the Tensorflow protobuf file (containing weights)
Extract all the weights and dump to disk (json encoded numbers)
Each node has one coresponding weight file if tensor_content
"""

import os.path

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from tensorflow.python.framework import tensor_util

import matplotlib.pyplot as plt
import json
import numpy as np

np.random.seed(0)

# parameter ==========================
wkdir = '../../data'
pb_filename = 'model.pb'
pbtxt_filename = 'model.pbtxt'

output_dir = os.path.join(wkdir, 'tensor_weights')
os.system("mkdir -p " + output_dir)

Using TensorFlow backend.


0

In [2]:
from tensorflow.python.platform import gfile

# For store the result file (before JSON encoding)
result = []

with tf.Session() as sess:
    # load model from pb file
    with gfile.FastGFile(wkdir+'/'+pb_filename,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    # write to tensorboard (check tensorboard for each op names)
    writer = tf.summary.FileWriter(wkdir+'/log/')
    writer.add_graph(sess.graph)
    writer.flush()
    writer.close()
    
    for i, node in enumerate(graph_def.node):
        # only care about those node has tensor_content
        if ('value' in node.attr):
            # skip training variables
            if ("training" in node.name):
                continue
    
            if (len(node.attr['value'].tensor.tensor_content)):
                tensor_np = tensor_util.MakeNdarray(node.attr['value'].tensor)
                tensor_size = tensor_np.shape
                
                # verify nditer's expected behavior
                print(tensor_size)
                nditer_verifier_data = []
                if (len(tensor_size) == 4):
                    for i in range(tensor_size[0]):
                        for j in range(tensor_size[1]):
                            for k in range(tensor_size[2]):
                                for w in range(tensor_size[3]):
                                    nditer_verifier_data.append(tensor_np[i,j,k,w].item())
                # save the data
                result = {}
                result['name'] = node.name
                result['shape'] = tensor_size
                result['tensor_content'] = []
                for i, x in enumerate(np.nditer(tensor_np)):
                    result['tensor_content'].append(x.item())
                    
                    # verify nditer data
                    if (len(tensor_size) == 4):
                        if (x.item() != nditer_verifier_data[i]):
                            print("[WARN] nditer verifier failed. ")
                
                # parse the name (each node name contains NN layer name + type_name[kernal/bias])
                layer_name = node.name.split("/")[0]
                type_name = node.name.split("/")[1]
                
                # kernal weights
                output_fname = os.path.join(output_dir, layer_name + "_" + type_name) + ".kw"
                print(output_fname)
                with open(output_fname, 'w') as outfile:  
                    json.dump(result, outfile)
                
    print("Completed. All kernal weights saved to disk.")

Instructions for updating:
Use tf.gfile.GFile.
(3, 3, 1, 32)
../../data/tensor_weights/conv2d_1_kernel.kw
(32,)
../../data/tensor_weights/conv2d_1_bias.kw
(3, 3, 32, 64)
../../data/tensor_weights/conv2d_2_kernel.kw
(64,)
../../data/tensor_weights/conv2d_2_bias.kw
(9216, 128)
../../data/tensor_weights/dense_1_kernel.kw
(128,)
../../data/tensor_weights/dense_1_bias.kw
(128, 10)
../../data/tensor_weights/dense_2_kernel.kw
(10,)
../../data/tensor_weights/dense_2_bias.kw
Completed. All kernal weights saved to disk.
